## 손글씨 숫자 인식 신경망 구축

In [1]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("1.12.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")

        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])

        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  404k    0  404k    0     0   435k      0 --:--:-- --:--:-- --:--:-- 3734k
  Cloning https://github.com/facebookresearch/pytorch3d.git (to revision stable) to /private/var/folders/yf/mmrj8c9d2zbfn3w8crhhf0_r0000gn/T/pip-req-build-t8pewzur
  Running command git clone -q https://github.com/facebookresearch/pytorch3d.git /private/var/folders/yf/mmrj8c9d2zbfn3w8crhhf0_r0000gn/T/pip-req-build-t8pewzur
  Running command git checkout -q 276c9a8acbfa715f5802f26ec9f4141bde26ecb4
  Resolved https://github.com/facebookresearch/pytorch3d.git to commit 276c9a8acbfa715f5802f26ec9f4141bde26ecb4
  Created wheel for pytorch3d: filename=pytorch3d-0.7.0-cp39-cp39-macosx_10_9_x86_64.whl size=807615 sha256=92f2921255b190499fbfaabb3bd42e102788c64579df8e9b275ef0d0afec5fa8

In [7]:
import os
import torch

# Util function for loading meshes
from pytorch3d.io import save_obj, load_obj

# Data structures and functions for rendering
from pytorch3d.structures import Meshes
from pytorch3d.utils import ico_sphere
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.loss import (
    chamfer_distance,
    mesh_edge_loss,
    mesh_laplacian_smoothing,
    mesh_normal_consistency
)

import numpy as np
from tqdm.notebook import tqdm
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

# Set the device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be slow!")



In [2]:
!wget https://dl.fbaipublicfiles.com/pytorch3d/data/dolphin/dolphin.obj

--2022-09-22 15:36:27--  https://dl.fbaipublicfiles.com/pytorch3d/data/dolphin/dolphin.obj
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 156763 (153K) [text/plain]
Saving to: 'dolphin.obj'

dolphin.obj         100%[===================>] 153.09K  --.-KB/s    in 0.1s    

2022-09-22 15:36:28 (1.33 MB/s) - 'dolphin.obj' saved [156763/156763]



In [5]:
# Load the dolphin mesh.
trg_obj = os.path.join('dolphin.obj')

In [6]:
verts, faces, aux = load_obj(trg_obj)
# verts:
# faces:
# aux:
faces_idx = faces.verts_idx.to(device)
verts = verts.to(device)

NameError: name 'load_obj' is not defined